# Simulate GUI
Simulate a simple GUI, show how queries are executed and how results are shown

In [1]:
from IPython.core.display import HTML
import requests
import json

## Load Metadata
Productive search engines use metadata, which should not be hardcoded.
We show a very simple metadata-management and read just the fields we configured as 'important_fields'
Productive metadata management will attach more information and is often managed in 
a separate application using an suitable database

In [11]:
# Load Metadata
path =  '/home/bfh/irsed/daten/FIFA/fifa_important_fields.csv'
important_fields = []
metadata = open(path,'r')
for line in metadata:
	important_fields.append(line.strip())

In [12]:
important_fields

['title',
 'description',
 'venues',
 'host_selection',
 'squads',
 'qualification',
 'goalscorers',
 'year']

## GUI fields
The GUI will normally contain different fields and field types.
We simulate this and define variables with hard-coded content

In [13]:
# Simulate the input - just set the field values without GUI
query = 'world'
search_options = 'important_fields'  # 'catch_all' or important_fields
requested_fields = 'important_fields' # 'catch_all' or important_fields

## Request
Based on this 'user input' we assemble our request

In [14]:
search_fields = []
if search_options == 'catch_all':
    search_fields = ['catch_all']
if search_options == 'important_fields':
    search_fields.extend(important_fields)

In [15]:
output_fields = []
if requested_fields == 'catch_all':
    output_fields = ['catch_all']
if requested_fields == 'important_fields':
    output_fields.extend(important_fields)

In [16]:
request = {
    "query": query,
    "params": {
      "fl": output_fields,
      "df": search_fields,
      "defType": "edismax",
      "indent": "true"
    }
}

request

{'query': 'world',
 'params': {'fl': ['title',
   'description',
   'venues',
   'host_selection',
   'squads',
   'qualification',
   'goalscorers',
   'year'],
  'df': ['title',
   'description',
   'venues',
   'host_selection',
   'squads',
   'qualification',
   'goalscorers',
   'year'],
  'defType': 'edismax',
  'indent': 'true'}}

## Execute the request

In [17]:
solr_url = 'http://localhost:8983/solr/'
collection = 'FIFA2'

response = requests.post(f"{solr_url}{collection}/select", json=request).json()  
print(request)
print(response)

{'query': 'world', 'params': {'fl': ['title', 'description', 'venues', 'host_selection', 'squads', 'qualification', 'goalscorers', 'year'], 'df': ['title', 'description', 'venues', 'host_selection', 'squads', 'qualification', 'goalscorers', 'year'], 'defType': 'edismax', 'indent': 'true'}}
{'responseHeader': {'status': 0, 'QTime': 4, 'params': {'json': '{"query": "world", "params": {"fl": ["title", "description", "venues", "host_selection", "squads", "qualification", "goalscorers", "year"], "df": ["title", "description", "venues", "host_selection", "squads", "qualification", "goalscorers", "year"], "defType": "edismax", "indent": "true"}}'}}, 'response': {'numFound': 22, 'start': 0, 'numFoundExact': True, 'docs': [{'year': '1930', 'title': '1930 FIFA World Cup', 'description': ['\nThe 1930 FIFA World Cup was the inaugural FIFA World Cup, the world championship for men\'s national football teams. It took place in Uruguay from 13 to 30 July 1930. FIFA, football\'s international governing

In [18]:
from IPython.core.display import HTML

html_str = '<h3>Anzahl Treffer: ' + str(response['response']['numFound']) + '</h3>'
html_str += '<h3>Dokumente</h3>'
HTML(html_str)


In [19]:
html_str = '<h3>Anzahl Treffer: ' + str(response['response']['numFound']) + '</h3>'
html_str += '<h3>Dokumente</h3>'

docs = response["response"]["docs"]
print(important_fields)
for doc in docs:
    for key in doc.keys():
        value = doc[key]
        html_str += '<br><b>' + key + '</b><br>'
        if isinstance(value,list):
            for elt in value:
                html_str += (elt.replace('\\n', '').replace(", '", "'"))
        else:
            html_str += (value.replace('\\n', '').replace(", '", "'"))
    html_str += '<hr/>'    
HTML(html_str)

['title', 'description', 'venues', 'host_selection', 'squads', 'qualification', 'goalscorers', 'year']
